In [14]:
import pickle
import numpy as np

In [15]:
with open('train_qa.txt', 'rb') as file:
    train_data = pickle.load(file)

In [16]:
with open('test_qa.txt', 'rb') as file:
    test_data = pickle.load(file)

In [17]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [18]:
test_data[0]

(['Mary',
  'got',
  'the',
  'milk',
  'there',
  '.',
  'John',
  'moved',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'John', 'in', 'the', 'kitchen', '?'],
 'no')

In [19]:
all_data = test_data + train_data

In [20]:
len(all_data)

11000

In [21]:
vocab = set()

for story,question,ans in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [22]:
vocab.add('no')
vocab.add('yes')

In [23]:
vocab


{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [44]:
vocab_len = len(vocab) + 1

In [24]:
# finding longest story length

all_story_lens = [len(data[0]) for data in all_data]

In [25]:
max_story_len = max(all_story_lens)

In [26]:
max_question_len = max([len(data[1]) for data in all_data])
max_question_len

6

In [30]:
import tensorflow

In [34]:
!pip install keras

In [36]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [37]:
tokenizer = Tokenizer(filters=[])

In [38]:
tokenizer.fit_on_texts(vocab)

In [39]:
tokenizer.word_index

{'there': 1,
 'football': 2,
 'picked': 3,
 'milk': 4,
 '.': 5,
 'left': 6,
 'put': 7,
 'is': 8,
 'to': 9,
 'john': 10,
 'sandra': 11,
 '?': 12,
 'grabbed': 13,
 'in': 14,
 'daniel': 15,
 'office': 16,
 'no': 17,
 'up': 18,
 'mary': 19,
 'the': 20,
 'went': 21,
 'dropped': 22,
 'kitchen': 23,
 'journeyed': 24,
 'yes': 25,
 'back': 26,
 'garden': 27,
 'down': 28,
 'took': 29,
 'bathroom': 30,
 'moved': 31,
 'hallway': 32,
 'bedroom': 33,
 'travelled': 34,
 'discarded': 35,
 'apple': 36,
 'got': 37}

In [40]:
train_story_text = []
train_question_text = []
train_answer_text = []

In [41]:
for story,question,answer in train_data:
    train_question_text.append(question)
    train_answer_text.append(answer)
    train_story_text.append(story)

In [42]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [43]:
train_story_seq

[[19, 31, 9, 20, 30, 5, 11, 24, 9, 20, 33, 5],
 [19,
  31,
  9,
  20,
  30,
  5,
  11,
  24,
  9,
  20,
  33,
  5,
  19,
  21,
  26,
  9,
  20,
  33,
  5,
  15,
  21,
  26,
  9,
  20,
  32,
  5],
 [19,
  31,
  9,
  20,
  30,
  5,
  11,
  24,
  9,
  20,
  33,
  5,
  19,
  21,
  26,
  9,
  20,
  33,
  5,
  15,
  21,
  26,
  9,
  20,
  32,
  5,
  11,
  21,
  9,
  20,
  23,
  5,
  15,
  21,
  26,
  9,
  20,
  30,
  5],
 [19,
  31,
  9,
  20,
  30,
  5,
  11,
  24,
  9,
  20,
  33,
  5,
  19,
  21,
  26,
  9,
  20,
  33,
  5,
  15,
  21,
  26,
  9,
  20,
  32,
  5,
  11,
  21,
  9,
  20,
  23,
  5,
  15,
  21,
  26,
  9,
  20,
  30,
  5,
  15,
  3,
  18,
  20,
  2,
  1,
  5,
  15,
  21,
  9,
  20,
  33,
  5],
 [19,
  31,
  9,
  20,
  30,
  5,
  11,
  24,
  9,
  20,
  33,
  5,
  19,
  21,
  26,
  9,
  20,
  33,
  5,
  15,
  21,
  26,
  9,
  20,
  32,
  5,
  11,
  21,
  9,
  20,
  23,
  5,
  15,
  21,
  26,
  9,
  20,
  30,
  5,
  15,
  3,
  18,
  20,
  2,
  1,
  5,
  15,
  21,
  9,
  20,
  3

In [46]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len = max_story_len, max_question_len = max_question_len):
    X = []
    Xq = []
    Y = []
    for story,query,answer in data:
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        y = np.zeros(len(word_index)+1)
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen= max_story_len), pad_sequences(Xq, maxlen= max_question_len), np.array(Y))

In [47]:
input_train, query_train, answer_train = vectorize_stories(train_data)

In [48]:
input_test, query_test, answer_test = vectorize_stories(test_data)

In [50]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Dropout, add, dot, concatenate, Activation, Permute, Input, Dense, LSTM

In [51]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [52]:
vocab_size = len(vocab)+1

In [53]:
#Input Encoder m

input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim = vocab_size, output_dim = 64))
input_encoder_m.add(Dropout(0.3))

In [54]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim = vocab_size, output_dim = max_question_len))
input_encoder_m.add(Dropout(0.3))

In [55]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim = vocab_size, output_dim = 64, input_length = max_question_len))
question_encoder.add(Dropout(0.3))

In [56]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [57]:
match = dot([input_encoded_m, question_encoded], axes=(2,2))
match = Activation('softmax')(match)

In [59]:
response = add([match, input_encoded_c])
response = Permute((2,1))(response)

In [60]:
answer = concatenate([response, question_encoded])

In [61]:
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [62]:
answer = LSTM(32)(answer)
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)

In [63]:
answer = Activation('softmax')(answer)

In [64]:
model = Model([input_sequence, question], answer)

In [65]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [66]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 156)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, None, 64)     2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
______________________________________________________________________________________________

In [68]:
history = model.fit([input_train, query_train], answer_train, batch_size=32, epochs=200, validation_data=([input_test,query_test], answer_test))

Epoch 1/200
313/313 [==============================] - 2s 7ms/step - loss: 0.6955 - accuracy: 0.4944 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 2/200
313/313 [==============================] - 2s 7ms/step - loss: 0.6947 - accuracy: 0.4989 - val_loss: 0.6956 - val_accuracy: 0.5030
Epoch 3/200
313/313 [==============================] - 2s 7ms/step - loss: 0.6948 - accuracy: 0.4972 - val_loss: 0.6955 - val_accuracy: 0.4970
Epoch 4/200
313/313 [==============================] - 2s 7ms/step - loss: 0.6944 - accuracy: 0.5027 - val_loss: 0.6932 - val_accuracy: 0.4600
Epoch 5/200
313/313 [==============================] - 2s 8ms/step - loss: 0.6947 - accuracy: 0.4995 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 6/200
313/313 [==============================] - 2s 7ms/step - loss: 0.6936 - accuracy: 0.5137 - val_loss: 0.6937 - val_accuracy: 0.4970
Epoch 7/200
313/313 [==============================] - 2s 8ms/step - loss: 0.6950 - accuracy: 0.4975 - val_loss: 0.6932 - val_accuracy: 0.5020

Epoch 58/200
313/313 [==============================] - 2s 6ms/step - loss: 0.2250 - accuracy: 0.9094 - val_loss: 0.4621 - val_accuracy: 0.8320
Epoch 59/200
313/313 [==============================] - 2s 6ms/step - loss: 0.2142 - accuracy: 0.9140 - val_loss: 0.4869 - val_accuracy: 0.8290
Epoch 60/200
313/313 [==============================] - 2s 6ms/step - loss: 0.2169 - accuracy: 0.9127 - val_loss: 0.5158 - val_accuracy: 0.8150
Epoch 61/200
313/313 [==============================] - 2s 6ms/step - loss: 0.2135 - accuracy: 0.9154 - val_loss: 0.4951 - val_accuracy: 0.8290
Epoch 62/200
313/313 [==============================] - 2s 6ms/step - loss: 0.2138 - accuracy: 0.9133 - val_loss: 0.4923 - val_accuracy: 0.8230
Epoch 63/200
313/313 [==============================] - 2s 7ms/step - loss: 0.2075 - accuracy: 0.9183 - val_loss: 0.5033 - val_accuracy: 0.8240
Epoch 64/200
313/313 [==============================] - 2s 6ms/step - loss: 0.2048 - accuracy: 0.9170 - val_loss: 0.5523 - val_accuracy:

Epoch 115/200
313/313 [==============================] - 2s 6ms/step - loss: 0.0985 - accuracy: 0.9626 - val_loss: 1.0723 - val_accuracy: 0.8030
Epoch 116/200
313/313 [==============================] - 2s 6ms/step - loss: 0.0930 - accuracy: 0.9659 - val_loss: 1.0894 - val_accuracy: 0.7980
Epoch 117/200
313/313 [==============================] - 2s 6ms/step - loss: 0.0978 - accuracy: 0.9633 - val_loss: 1.1252 - val_accuracy: 0.8030
Epoch 118/200
313/313 [==============================] - 2s 6ms/step - loss: 0.0995 - accuracy: 0.9630 - val_loss: 1.1117 - val_accuracy: 0.7980
Epoch 119/200
313/313 [==============================] - 2s 6ms/step - loss: 0.0932 - accuracy: 0.9653 - val_loss: 1.1253 - val_accuracy: 0.7990
Epoch 120/200
313/313 [==============================] - 2s 6ms/step - loss: 0.0994 - accuracy: 0.9634 - val_loss: 1.0866 - val_accuracy: 0.8010
Epoch 121/200
313/313 [==============================] - 2s 6ms/step - loss: 0.0915 - accuracy: 0.9661 - val_loss: 1.0995 - val_ac

313/313 [==============================] - 2s 7ms/step - loss: 0.0580 - accuracy: 0.9792 - val_loss: 1.5077 - val_accuracy: 0.7920
Epoch 172/200
313/313 [==============================] - 2s 7ms/step - loss: 0.0617 - accuracy: 0.9779 - val_loss: 1.5723 - val_accuracy: 0.7900
Epoch 173/200
313/313 [==============================] - 2s 7ms/step - loss: 0.0658 - accuracy: 0.9792 - val_loss: 1.5263 - val_accuracy: 0.7990
Epoch 174/200
313/313 [==============================] - 2s 7ms/step - loss: 0.0591 - accuracy: 0.9789 - val_loss: 1.6043 - val_accuracy: 0.7920
Epoch 175/200
313/313 [==============================] - 2s 7ms/step - loss: 0.0574 - accuracy: 0.9815 - val_loss: 1.5435 - val_accuracy: 0.7910
Epoch 176/200
313/313 [==============================] - 2s 7ms/step - loss: 0.0562 - accuracy: 0.9781 - val_loss: 1.5836 - val_accuracy: 0.7880
Epoch 177/200
313/313 [==============================] - 2s 7ms/step - loss: 0.0581 - accuracy: 0.9795 - val_loss: 1.5724 - val_accuracy: 0.7940

In [69]:
model.save('chat_bot200.h5')

In [70]:
pred_result = model.predict(([input_test, query_test]))

In [71]:
pred_result

array([[2.4283502e-20, 2.5459378e-20, 2.2830174e-20, ..., 2.3256669e-20,
        2.5143386e-20, 2.6307056e-20],
       [6.5104413e-19, 7.2556944e-19, 6.0091270e-19, ..., 6.0710383e-19,
        7.2506589e-19, 6.9327082e-19],
       [2.5889223e-18, 2.4795624e-18, 2.7122460e-18, ..., 2.6169025e-18,
        2.5356926e-18, 2.4717526e-18],
       ...,
       [1.0988988e-19, 1.2245922e-19, 1.1063605e-19, ..., 1.0256313e-19,
        1.2249286e-19, 1.1075683e-19],
       [9.5191407e-18, 9.6398777e-18, 1.0218821e-17, ..., 1.0135469e-17,
        8.8649658e-18, 9.8946021e-18],
       [1.1755037e-18, 1.2029484e-18, 1.2836474e-18, ..., 1.3400829e-18,
        1.1297982e-18, 1.1855144e-18]], dtype=float32)

In [72]:
val_max = np.argmax(pred_result[0])

In [75]:
for key,value in tokenizer.word_index.items():
    if value == val_max:
        k = key

In [76]:
k

'no'

In [77]:
pred_result[0][val_max]

1.0

In [78]:
my_story = "John left the kitchen . Sandra dropped the football in garden"

In [79]:
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'garden']

In [80]:
my_question = "Is the football in the garden ?"

In [81]:
my_question.split()

['Is', 'the', 'football', 'in', 'the', 'garden', '?']

In [82]:
mydata = [(my_story.split(), my_question.split(), "yes")] 

In [83]:
mydata

[(['John',
   'left',
   'the',
   'kitchen',
   '.',
   'Sandra',
   'dropped',
   'the',
   'football',
   'in',
   'garden'],
  ['Is', 'the', 'football', 'in', 'the', 'garden', '?'],
  'yes')]

In [84]:
my_story, my_ques, my_ans = vectorize_stories(mydata)

In [85]:
my_story

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 10,  6, 20, 23,  5, 11, 22, 20,  2, 14, 27]], dtype=int32)

In [86]:
my_ques

array([[20,  2, 14, 20, 27, 12]], dtype=int32)

In [87]:
my_ans

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])

In [89]:
predicted_result = model.predict(([my_story, my_ques]))

In [90]:
predicted_result

array([[3.3981598e-12, 3.3810014e-12, 3.1490724e-12, 3.2038775e-12,
        3.2419655e-12, 2.8587332e-12, 3.2471699e-12, 2.8395882e-12,
        2.6423132e-12, 3.2386403e-12, 3.0055997e-12, 3.4053487e-12,
        2.8521215e-12, 3.6883933e-12, 3.0573304e-12, 3.0986424e-12,
        3.8974214e-12, 9.9949110e-01, 3.4016485e-12, 3.2302008e-12,
        3.0900668e-12, 3.5130575e-12, 3.1461426e-12, 4.1270793e-12,
        3.3395151e-12, 5.0886901e-04, 3.3771601e-12, 3.7421208e-12,
        3.8181402e-12, 3.2836359e-12, 3.3343280e-12, 3.1894517e-12,
        3.1273744e-12, 3.2159878e-12, 3.0959959e-12, 3.5374787e-12,
        2.9194347e-12, 2.9387090e-12]], dtype=float32)

In [91]:
val_max = np.argmax(predicted_result[0])

In [92]:
for key,value in tokenizer.word_index.items():
    if value == val_max:
        k = key

In [93]:
k

'no'